In [24]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer

In [29]:
# Read the text from the file
with open('dataset.txt', 'r', encoding='utf-8') as file:
    faqs = file.read()

In [30]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([faqs])
len(tokenizer.word_index)

1088

In [31]:
#tokenizer.word_index

In [43]:
#converting the dataset for applying supervised algorithm
input_sequences = []
for sentence in faqs.split('\n'):
        tokenized_sentence = tokenizer.texts_to_sequences([sentence])[0]

        for i in range(1, len(tokenized_sentence)):
            n_gram_sequence = tokenized_sentence[:i+1]
            input_sequences.append(n_gram_sequence)

In [44]:
input_sequences

[[567, 568],
 [387, 299],
 [387, 299, 35],
 [387, 299, 35, 569],
 [387, 299, 35, 569, 12],
 [387, 299, 35, 569, 12, 388],
 [387, 299, 35, 569, 12, 388, 24],
 [387, 299, 35, 569, 12, 388, 24, 48],
 [387, 299, 35, 569, 12, 388, 24, 48, 1],
 [387, 299, 35, 569, 12, 388, 24, 48, 1, 193],
 [570, 102],
 [570, 102, 571],
 [570, 102, 571, 89],
 [570, 102, 571, 89, 572],
 [570, 102, 571, 89, 572, 573],
 [570, 102, 571, 89, 572, 573, 574],
 [570, 102, 571, 89, 572, 573, 574, 1],
 [570, 102, 571, 89, 572, 573, 574, 1, 238],
 [570, 102, 571, 89, 572, 573, 574, 1, 238, 575],
 [576, 102],
 [576, 102, 577],
 [576, 102, 577, 389],
 [576, 102, 577, 389, 390],
 [576, 102, 577, 389, 390, 578],
 [576, 102, 577, 389, 390, 578, 194],
 [576, 102, 577, 389, 390, 578, 194, 579],
 [576, 102, 577, 389, 390, 578, 194, 579, 580],
 [576, 102, 577, 389, 390, 578, 194, 579, 580, 6],
 [576, 102, 577, 389, 390, 578, 194, 579, 580, 6, 581],
 [576, 102, 577, 389, 390, 578, 194, 579, 580, 6, 581, 239],
 [391, 392],
 [391,

In [48]:
#applying zero padding to have an equal number in every vector
print(f"Total sequences: {len(input_sequences)}")
max_len = max([len(x) for x in input_sequences])


Total sequences: 3401


In [49]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
pad_sequences(input_sequences, maxlen = max_len, padding = 'pre')

array([[   0,    0,    0, ...,    0,  567,  568],
       [   0,    0,    0, ...,    0,  387,  299],
       [   0,    0,    0, ...,  387,  299,   35],
       ...,
       [   0,    0,    0, ...,    0,  284,  195],
       [   0,    0,    0, ...,  284,  195,  380],
       [   0,    0,    0, ...,  195,  380, 1088]], dtype=int32)